In [4]:
class Vendas():
    def __init__(self, regiao, estado, municipio, revenda, 
                 produto, dt_coleta, valor_compra, 
                 valor_venda, unidade_medida, bandeira):
        self.regiao = regiao
        self.estado = estado
        self.municipio = municipio
        self.revenda = revenda
        self.produto = produto
        self.dt_coleta = dt_coleta
        #self.mes = mes
        #self.ano = ano
        self.valor_compra = valor_compra
        self.valor_venda = valor_venda
        self.unidade_medida = unidade_medida
        self.bandeira = bandeira

    def get_regiao(self):
        return self.regiao

    def get_estado(self):
        return self.estado

    def get_municipio(self):
        return self.municipio

    def get_revenda(self):
        return self.revenda

    def get_produto(self):
        return self.produto    
        
    def get_dt_coleta(self):
        return self.dt_coleta
    
    def get_mes(self):
        return self.mes
    
    def get_ano(self):
        return self.ano
    
    def get_valor_compra(self):
        return self.valor_compra    
    
    def get_unidade_medida(self):
        return self.unidade_medida        
    
    def get_bandeira(self):
        return self.bandeira     

In [8]:
def load_vendas_info_file(fname):
    with open(fname) as f:
        next(f)
        for line in f:    
            cs = Cruise_ship(**str(line).strip().split('\t'))
    f.close()

In [9]:
load_vendas_info_file('2016-2_CA.csv')
load_vendas_info_file('2016-2_CA.csv')
load_vendas_info_file('2016-2_CA.csv')
load_vendas_info_file('2016-2_CA.csv')

UnicodeDecodeError: 'utf-8' codec can't decode byte 0xe3 in position 4: invalid continuation byte

In [ ]:
from cassandra.cluster import Cluster
from cassandra.auth import PlainTextAuthProvider

CASSANDRA_USER = 'cassandra'
CASSANDRA_PASSWD = 'password'

def create_connection(nodeIP, keyspaceName):
    """
    Function to create a connection object
    """

    client = Connection()
    client.connect([nodeIP]) #Observar que o nodeIP é uma lista e não somente um único valor
    client.set_keyspace(keyspaceName)
    return client


class Connection:
    session = None
    keyspace = ""

    def connect(self, nodes):
        auth_provider = PlainTextAuthProvider(username=str(CASSANDRA_USER), password=str(CASSANDRA_PASSWD))
        cluster = Cluster(nodes, auth_provider=auth_provider)
        metadata = cluster.metadata
        self.session = cluster.connect()

    def get_session():
        return self.session

    def set_keyspace(self, keyspace):
        self.keyspace = keyspace

    def get_keyspace(self):
        return self.keyspace

    def close(self):
        self.session.cluster.shutdown()
        self.session.shutdown()

    def query_schema(self, table):
        table_keyspace = self.keyspace+"."+table
        query = " SELECT * FROM "+ table_keyspace +";"
        result_set = self.session.execute(query)
        return result_set

In [ ]:
#Classe pai das conexões
class CommonCassandraDB():

    def __init__(self):
        self.client = None

    def set_cassandra_client(self, nodeIP, keyspaceName):
        self.client = create_connection(nodeIP, keyspaceName)

    def close_connection(self):
        self.client.close()

In [ ]:
class Cruzeiros_cassandra(CommonCassandraDB):
    
    def __init__(self, nodeIP, keyspace):
        self.cassandra_table_reference = 'vendas' #nome da Tabela do Cassandra

        self.set_cassandra_client(nodeIP, keyspace)
    
    def get_full_table(self):
        table = self.client.get_keyspace() + "." + self.cassandra_table_reference
        sql_query = """
                    SELECT *
                    FROM @$$$$$$$$$$$$@
                """
        sql_query = sql_query.replace("@$$$$$$$$$$$$@", table)
        result_set = self.client.session.execute(
            sql_query
        )
        return result_set 

    def show_row(self, row):
        print(row.ship_name)
        print(row.cruise_line)
        print(str(row.age))
        print(str(row.tonnage))

    def insert_cruzeiros_cruise_line(self, cruise_ship):
        try:
            table = self.client.get_keyspace() + "." + 'vendas'
            sql_query = """
            INSERT INTO @$$$$$$$$$$$$@
            (
                regiao, 
                estado, 
                municipio, 
                revenda, 
                produto, 
                dt_coleta, 
                mes,
                ano,
                valor_compra, 
                valor_venda, 
                unidade_medida, 
                bandeira
            )
            VALUES (?,?,?,?,?,?,?,?,?,?,?,?);
            """
            sql_query = sql_query.replace("@$$$$$$$$$$$$@", table)
            bound_statement = self.client.session.prepare(sql_query)
            self.client.session.execute( bound_statement.bind((
                    str(cruise_ship.get_ship_name()),
                    str(cruise_ship.get_cruise_line()),
                    int(cruise_ship.get_age()),
                    float(cruise_ship.get_tonnage()),
                
                
                regiao, 
                estado, 
                municipio, 
                revenda, 
                produto, 
                dt_coleta, 
                mes,
                ano,
                valor_compra, 
                valor_venda, 
                unidade_medida, 
                bandeira                
                
                
                    ))
                )
            return True
        except:
            raise
        return False
        
    def insert(self, cruise_ship):
        try:
            table = self.client.get_keyspace() + "." + self.cassandra_table_reference
            sql_query = """
            INSERT INTO @$$$$$$$$$$$$@
            (
                ship_name,
                cruise_line,
                age,
                tonnage
            )
            VALUES (?,?,?,?);
            """
            sql_query = sql_query.replace("@$$$$$$$$$$$$@", table)
            bound_statement = self.client.session.prepare(sql_query)
            self.client.session.execute( bound_statement.bind((
                    str(cruise_ship.get_ship_name()),
                    str(cruise_ship.get_cruise_line()),
                    int(cruise_ship.get_age()),
                    float(cruise_ship.get_tonnage()),
                    ))
                )
            # Insere na Tabela cruzeiros_cruise_line
            self.insert_cruzeiros_cruise_line(cruise_ship)
            return True
        except:
            raise
        return False
    
    def get_age(self, ship_name):
        table = self.client.get_keyspace() + "." + self.cassandra_table_reference
        sql_query = """
                    SELECT age
                    FROM @$$$$$$$$$$$$@
                """
        sql_query = sql_query.replace("@$$$$$$$$$$$$@", table)
        #Cria o filtro para ship_name
        where_sql_q_ship_name = " WHERE ship_name = "
        where_sql_q_ship_name = ''.join([where_sql_q_ship_name,"'", ship_name, "'"])
        sql_query = ''.join([sql_query, where_sql_q_ship_name])
        
        result_set = self.client.session.execute(
            sql_query
        )
        return result_set 
    
    def get_cruise_line(self, ship_name):
        table = self.client.get_keyspace() + "." + self.cassandra_table_reference
        sql_query = """
                    SELECT cruise_line
                    FROM @$$$$$$$$$$$$@
                """
        sql_query = sql_query.replace("@$$$$$$$$$$$$@", table)
        #Cria o filtro para ship_name
        where_sql_q_ship_name = " WHERE ship_name = "
        where_sql_q_ship_name = ''.join([where_sql_q_ship_name,"'", ship_name, "'"])
        sql_query = ''.join([sql_query, where_sql_q_ship_name])
        
        result_set = self.client.session.execute(
            sql_query
        )
        return result_set 
    
    def get_distinct_cruise_line(self):
        table = self.client.get_keyspace() + "." + "cruzeiros_cruise_line"
        sql_query = """
                    SELECT DISTINCT cruise_line
                    FROM @$$$$$$$$$$$$@
                """
        sql_query = sql_query.replace("@$$$$$$$$$$$$@", table)
        result_set = self.client.session.execute(
            sql_query
        )
        return result_set 
    
    def get_ship_name_cruise_line_with_removing(self,l_cruise_line_filtered):
        table = self.client.get_keyspace() + "." + "cruzeiros_cruise_line"
        sql_query = """
                    SELECT ship_name, cruise_line
                    FROM @$$$$$$$$$$$$@
                """
        sql_query = sql_query.replace("@$$$$$$$$$$$$@", table)
        #Cria o filtro para cruise_line
        where_sql_q_cruise_line = " WHERE cruise_line IN ( "
        a = 1
        for cruise_line_filtered in l_cruise_line_filtered:
            if a > 1:
                where_sql_q_cruise_line = ''.join([where_sql_q_cruise_line,","])
            where_sql_q_cruise_line = ''.join([where_sql_q_cruise_line,"'", cruise_line_filtered, "'"])
            a += 1
        where_sql_q_cruise_line = ''.join([where_sql_q_cruise_line, ' ) ' ])    
        sql_query = ''.join([sql_query, where_sql_q_cruise_line])
        result_set = self.client.session.execute(
            sql_query
        )
        return result_set 